Strategy Logic and Signal Generation

In [ ]:
import pandas as pd
import numpy as np

def generate_vol_norm_rsi_signals(df, rsi_entry=30, rsi_exit=50, atr_multiplier=1.0, vix_threshold=40):
    """
    Generates trading signals based on the Volatility-Normalised RSI strategy.
    Returns a DataFrame with a 'signal' column (1 for long, -1 for short, 0 for flat).
    """
    signals = pd.DataFrame(index=df.index)
    signals['signal'] = 0

    # Entry Conditions
    long_entry_condition = (df['rsi'] < rsi_entry) & \
                           (df['spy_close'] < (df['spy_close'].shift(1) - (atr_multiplier * df['atr'])))
    
    short_entry_condition = (df['rsi'] > (100 - rsi_entry)) & \
                            (df['spy_close'] > (df['spy_close'].shift(1) + (atr_multiplier * df['atr'])))

    # Exit Conditions
    long_exit_condition = df['rsi'] > rsi_exit
    short_exit_condition = df['rsi'] < (100 - rsi_exit)

    # State machine to hold positions
    position = 0
    for i in range(len(df)):
        if position == 0: # If flat
            if long_entry_condition.iloc[i]:
                position = 1
            elif short_entry_condition.iloc[i]:
                position = -1
        elif position == 1: # If long
            if long_exit_condition.iloc[i]:
                position = 0
        elif position == -1: # If short
            if short_exit_condition.iloc[i]:
                position = 0
        signals['signal'].iloc[i] = position

    # VIX Regime Filter - Fixed the assignment
    vix_high_mask = df['vix'] > vix_threshold
    signals.loc[vix_high_mask, 'signal'] = 0
    
    return signals['signal']

# Example usage (in the same script or imported elsewhere):
if __name__ == '__main__':
    # Import the function from the previous script
    def calculate_features(df):
        """Engineers all necessary features from the raw OHLCV data."""
        df = df.copy()
        
        # 1. Relative Strength Index (RSI) - Fixed to handle division by zero
        delta = df['spy_close'].diff()  # Use spy_close instead of 'Close'
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        
        # Handle division by zero and NaN values
        rs = np.where((loss == 0) | (loss.isna()), np.nan, gain / loss)
        df['rsi'] = 100 - (100 / (1 + rs))
        
        # 2. Bollinger Bands
        df['sma_20'] = df['spy_close'].rolling(window=20).mean()
        df['std_20'] = df['spy_close'].rolling(window=20).std()
        df['bollinger_upper'] = df['sma_20'] + (df['std_20'] * 2)
        df['bollinger_lower'] = df['sma_20'] - (df['std_20'] * 2)

        # 3. Rolling Z-Score
        df['z_score_20'] = (df['spy_close'] - df['sma_20']) / df['std_20']

        # 4. Average True Range (ATR) - Fixed column references
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['spy_close'].shift())  # Use spy_close
        low_close = np.abs(df['Low'] - df['spy_close'].shift())    # Use spy_close
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        df['atr'] = tr.rolling(window=14).mean()
        
        return df

    def get_market_data(start_date="2005-01-01"):
        """Downloads SPY and VIX data and engineers features."""
        import yfinance as yf
        import numpy as np
        
        spy_data = yf.download('SPY', start=start_date, auto_adjust=True)
        vix_data = yf.download('^VIX', start=start_date, auto_adjust=True)

        # Reset any potential multi-level indexes and ensure clean column structure
        if isinstance(spy_data.columns, pd.MultiIndex):
            spy_data.columns = spy_data.columns.droplevel(1)  # Remove the second level if it exists
        if isinstance(vix_data.columns, pd.MultiIndex):
            vix_data.columns = vix_data.columns.droplevel(1)  # Remove the second level if it exists
        
        # Select only the columns we need
        spy_data = spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        vix_data = vix_data[['Close']].copy()
        vix_data = vix_data.rename(columns={'Close': 'vix'})
        
        # Align the date indices properly
        common_dates = spy_data.index.intersection(vix_data.index)
        spy_data = spy_data.loc[common_dates]
        vix_data = vix_data.loc[common_dates]
        
        # Combine into a single DataFrame using merge
        df = spy_data.copy()
        df['vix'] = vix_data['vix']
        df.rename(columns={'Close': 'spy_close'}, inplace=True)  # Rename for clarity

        # Calculate features using the function
        df = calculate_features(df)
        
        # Calculate daily returns for the backtest
        df['daily_return'] = df['spy_close'].pct_change()

        return df.dropna()

    market_data = get_market_data()
    # Use default parameters for now
    signals = generate_vol_norm_rsi_signals(market_data)
    
    # Check how many signals were generated
    print("Signal distribution:")
    print(signals.value_counts())

    # Plot RSI with signals
    import matplotlib.pyplot as plt
    
    market_data['signal'] = signals
    
    plt.figure(figsize=(15, 7))
    plt.plot(market_data.index, market_data['spy_close'], label='SPY Close')
    plt.scatter(market_data[market_data['signal'] == 1].index, market_data[market_data['signal'] == 1]['spy_close'], marker='^', color='g', s=100, label='Buy Signal')
    plt.scatter(market_data[market_data['signal'] == -1].index, market_data[market_data['signal'] == -1]['spy_close'], marker='v', color='r', s=100, label='Short Signal')
    plt.legend()
    plt.title('SPY with Volatility-Normalised RSI Signals')
    plt.show()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
C:\Users\USER\AppData\Local\Temp\ipykernel_24820\4097696075.py:37: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

Signal distribution:
signal
 0    3603
-1    1143
 1     457
Name: count, dtype: int64


# Price Target Exit (Reversion to the Mean)

In [ ]:
import pandas as pd
import numpy as np

def generate_sma_exit_signals(df, rsi_entry=30, atr_multiplier=1.5, vix_threshold=40):
    """
    Generates signals with an exit condition based on crossing the 20-day SMA.
    """
    signals = pd.DataFrame(index=df.index)
    signals['signal'] = 0.0
    position = 0  # 0: flat, 1: long, -1: short

    # --- ENTRY CONDITIONS (Same as before) ---
    long_entry = (df['rsi'] < rsi_entry) & \
                 (df['spy_close'] < (df['spy_close'].shift(1) - (atr_multiplier * df['atr'])))
    
    short_entry = (df['rsi'] > (100 - rsi_entry)) & \
                  (df['spy_close'] > (df['spy_close'].shift(1) + (atr_multiplier * df['atr'])))

    for i in range(1, len(df)):
        # --- EXIT CONDITIONS ---
        # Exit long if price crosses above the 20-day SMA
        if position == 1 and (df['spy_close'].iloc[i] > df['sma_20'].iloc[i]):
            position = 0
        # Exit short if price crosses below the 20-day SMA
        elif position == -1 and (df['spy_close'].iloc[i] < df['sma_20'].iloc[i]):
            position = 0
        
        # --- ENTRY LOGIC ---
        if position == 0:
            if long_entry.iloc[i]:
                position = 1
            elif short_entry.iloc[i]:
                position = -1
        
        signals['signal'].iloc[i] = position

    # VIX Regime Filter
    signals.loc[df['vix'] > vix_threshold, 'signal'] = 0
    
    return signals['signal']

# Example usage (in the same script or imported elsewhere):
if __name__ == '__main__':
    # Import the function from the previous script
    def calculate_features(df):
        """Engineers all necessary features from the raw OHLCV data."""
        df = df.copy()
        
        # 1. Relative Strength Index (RSI) - Fixed to handle division by zero
        delta = df['spy_close'].diff()  # Use spy_close instead of 'Close'
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        
        # Handle division by zero and NaN values
        rs = np.where((loss == 0) | (loss.isna()), np.nan, gain / loss)
        df['rsi'] = 100 - (100 / (1 + rs))
        
        # 2. Bollinger Bands
        df['sma_20'] = df['spy_close'].rolling(window=20).mean()
        df['std_20'] = df['spy_close'].rolling(window=20).std()
        df['bollinger_upper'] = df['sma_20'] + (df['std_20'] * 2)
        df['bollinger_lower'] = df['sma_20'] - (df['std_20'] * 2)

        # 3. Rolling Z-Score
        df['z_score_20'] = (df['spy_close'] - df['sma_20']) / df['std_20']

        # 4. Average True Range (ATR) - Fixed column references
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['spy_close'].shift())  # Use spy_close
        low_close = np.abs(df['Low'] - df['spy_close'].shift())    # Use spy_close
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        df['atr'] = tr.rolling(window=14).mean()
        
        return df

    def get_market_data(start_date="2005-01-01"):
        """Downloads SPY and VIX data and engineers features."""
        import yfinance as yf
        import numpy as np
        
        spy_data = yf.download('SPY', start=start_date, auto_adjust=True)
        vix_data = yf.download('^VIX', start=start_date, auto_adjust=True)

        # Reset any potential multi-level indexes and ensure clean column structure
        if isinstance(spy_data.columns, pd.MultiIndex):
            spy_data.columns = spy_data.columns.droplevel(1)  # Remove the second level if it exists
        if isinstance(vix_data.columns, pd.MultiIndex):
            vix_data.columns = vix_data.columns.droplevel(1)  # Remove the second level if it exists
        
        # Select only the columns we need
        spy_data = spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        vix_data = vix_data[['Close']].copy()
        vix_data = vix_data.rename(columns={'Close': 'vix'})
        
        # Align the date indices properly
        common_dates = spy_data.index.intersection(vix_data.index)
        spy_data = spy_data.loc[common_dates]
        vix_data = vix_data.loc[common_dates]
        
        # Combine into a single DataFrame using merge
        df = spy_data.copy()
        df['vix'] = vix_data['vix']
        df.rename(columns={'Close': 'spy_close'}, inplace=True)  # Rename for clarity

        # Calculate features using the function
        df = calculate_features(df)
        
        # Calculate daily returns for the backtest
        df['daily_return'] = df['spy_close'].pct_change()

        return df.dropna()

    market_data = get_market_data()
    # Use default parameters for now
    signals = generate_sma_exit_signals(market_data)
    
    # Check how many signals were generated
    print("Signal distribution:")
    print(signals.value_counts())

    # Plot RSI with signals
    import matplotlib.pyplot as plt
    
    market_data['signal'] = signals
    
    plt.figure(figsize=(15, 7))
    plt.plot(market_data.index, market_data['spy_close'], label='SPY Close')
    plt.scatter(market_data[market_data['signal'] == 1].index, market_data[market_data['signal'] == 1]['spy_close'], marker='^', color='g', s=100, label='Buy Signal')
    plt.scatter(market_data[market_data['signal'] == -1].index, market_data[market_data['signal'] == -1]['spy_close'], marker='v', color='r', s=100, label='Short Signal')
    plt.legend()
    plt.title('SPY with Volatility-Normalised RSI Signals')
    plt.show()

# Time-Based Exit

In [ ]:
import pandas as pd
import numpy as np

def generate_time_exit_signals(df, rsi_entry=30, atr_multiplier=1.5, vix_threshold=40, exit_days=5):
    """
    Generates signals with an exit condition based on a fixed holding period.
    """
    signals = pd.DataFrame(index=df.index)
    signals['signal'] = 0.0
    position = 0  # 0: flat, 1: long, -1: short
    entry_bar = 0  # To track the bar number of the entry

    # --- ENTRY CONDITIONS (Same as before) ---
    long_entry = (df['rsi'] < rsi_entry) & \
                 (df['spy_close'] < (df['spy_close'].shift(1) - (atr_multiplier * df['atr'])))
    
    short_entry = (df['rsi'] > (100 - rsi_entry)) & \
                  (df['spy_close'] > (df['spy_close'].shift(1) + (atr_multiplier * df['atr'])))

    for i in range(1, len(df)):
        # --- EXIT CONDITION ---
        # Exit if holding period has been reached
        if position != 0 and (i - entry_bar >= exit_days):
            position = 0
        
        # --- ENTRY LOGIC ---
        if position == 0:
            if long_entry.iloc[i]:
                position = 1
                entry_bar = i  # Log the entry bar
            elif short_entry.iloc[i]:
                position = -1
                entry_bar = i  # Log the entry bar
        
        signals['signal'].iloc[i] = position

    # VIX Regime Filter
    signals.loc[df['vix'] > vix_threshold, 'signal'] = 0
    
    return signals['signal']

# Example usage (in the same script or imported elsewhere):
if __name__ == '__main__':
    # Import the function from the previous script
    def calculate_features(df):
        """Engineers all necessary features from the raw OHLCV data."""
        df = df.copy()
        
        # 1. Relative Strength Index (RSI) - Fixed to handle division by zero
        delta = df['spy_close'].diff()  # Use spy_close instead of 'Close'
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        
        # Handle division by zero and NaN values
        rs = np.where((loss == 0) | (loss.isna()), np.nan, gain / loss)
        df['rsi'] = 100 - (100 / (1 + rs))
        
        # 2. Bollinger Bands
        df['sma_20'] = df['spy_close'].rolling(window=20).mean()
        df['std_20'] = df['spy_close'].rolling(window=20).std()
        df['bollinger_upper'] = df['sma_20'] + (df['std_20'] * 2)
        df['bollinger_lower'] = df['sma_20'] - (df['std_20'] * 2)

        # 3. Rolling Z-Score
        df['z_score_20'] = (df['spy_close'] - df['sma_20']) / df['std_20']

        # 4. Average True Range (ATR) - Fixed column references
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['spy_close'].shift())  # Use spy_close
        low_close = np.abs(df['Low'] - df['spy_close'].shift())    # Use spy_close
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        df['atr'] = tr.rolling(window=14).mean()
        
        return df

    def get_market_data(start_date="2005-01-01"):
        """Downloads SPY and VIX data and engineers features."""
        import yfinance as yf
        import numpy as np
        
        spy_data = yf.download('SPY', start=start_date, auto_adjust=True)
        vix_data = yf.download('^VIX', start=start_date, auto_adjust=True)

        # Reset any potential multi-level indexes and ensure clean column structure
        if isinstance(spy_data.columns, pd.MultiIndex):
            spy_data.columns = spy_data.columns.droplevel(1)  # Remove the second level if it exists
        if isinstance(vix_data.columns, pd.MultiIndex):
            vix_data.columns = vix_data.columns.droplevel(1)  # Remove the second level if it exists
        
        # Select only the columns we need
        spy_data = spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        vix_data = vix_data[['Close']].copy()
        vix_data = vix_data.rename(columns={'Close': 'vix'})
        
        # Align the date indices properly
        common_dates = spy_data.index.intersection(vix_data.index)
        spy_data = spy_data.loc[common_dates]
        vix_data = vix_data.loc[common_dates]
        
        # Combine into a single DataFrame using merge
        df = spy_data.copy()
        df['vix'] = vix_data['vix']
        df.rename(columns={'Close': 'spy_close'}, inplace=True)  # Rename for clarity

        # Calculate features using the function
        df = calculate_features(df)
        
        # Calculate daily returns for the backtest
        df['daily_return'] = df['spy_close'].pct_change()

        return df.dropna()

    market_data = get_market_data()
    # Use default parameters for now
    signals = generate_time_exit_signals(market_data)
    
    # Check how many signals were generated
    print("Signal distribution:")
    print(signals.value_counts())

    # Plot RSI with signals
    import matplotlib.pyplot as plt
    
    market_data['signal'] = signals
    
    plt.figure(figsize=(15, 7))
    plt.plot(market_data.index, market_data['spy_close'], label='SPY Close')
    plt.scatter(market_data[market_data['signal'] == 1].index, market_data[market_data['signal'] == 1]['spy_close'], marker='^', color='g', s=100, label='Buy Signal')
    plt.scatter(market_data[market_data['signal'] == -1].index, market_data[market_data['signal'] == -1]['spy_close'], marker='v', color='r', s=100, label='Short Signal')
    plt.legend()
    plt.title('SPY with Volatility-Normalised RSI Signals')
    plt.show()

# ATR-Based Profit Target & Stop Loss

In [ ]:
import pandas as pd
import numpy as np

def generate_atr_target_signals(df, rsi_entry=30, atr_multiplier=1.5, vix_threshold=40, stop_loss_mult=1.5, profit_target_mult=2.0):
    """
    Generates signals with fixed stop loss and profit targets based on entry ATR.
    """
    signals = pd.DataFrame(index=df.index)
    signals['signal'] = 0.0
    position = 0  # 0: flat, 1: long, -1: short
    entry_price = 0.0
    stop_loss_price = 0.0
    profit_target_price = 0.0

    # --- ENTRY CONDITIONS (Same as before) ---
    long_entry = (df['rsi'] < rsi_entry) & \
                 (df['spy_close'] < (df['spy_close'].shift(1) - (atr_multiplier * df['atr'])))
    
    short_entry = (df['rsi'] > (100 - rsi_entry)) & \
                  (df['spy_close'] > (df['spy_close'].shift(1) + (atr_multiplier * df['atr'])))

    for i in range(1, len(df)):
        # --- EXIT CONDITIONS ---
        if position == 1: # Long position
            if df['spy_close'].iloc[i] <= stop_loss_price or df['spy_close'].iloc[i] >= profit_target_price:
                position = 0
        elif position == -1: # Short position
            if df['spy_close'].iloc[i] >= stop_loss_price or df['spy_close'].iloc[i] <= profit_target_price:
                position = 0
        
        # --- ENTRY LOGIC ---
        if position == 0:
            if long_entry.iloc[i]:
                position = 1
                entry_price = df['spy_close'].iloc[i]
                atr_at_entry = df['atr'].iloc[i]
                stop_loss_price = entry_price - (atr_at_entry * stop_loss_mult)
                profit_target_price = entry_price + (atr_at_entry * profit_target_mult)
            elif short_entry.iloc[i]:
                position = -1
                entry_price = df['spy_close'].iloc[i]
                atr_at_entry = df['atr'].iloc[i]
                stop_loss_price = entry_price + (atr_at_entry * stop_loss_mult)
                profit_target_price = entry_price - (atr_at_entry * profit_target_mult)
        
        signals['signal'].iloc[i] = position

    # VIX Regime Filter
    signals.loc[df['vix'] > vix_threshold, 'signal'] = 0
    
    return signals['signal']

# Example usage (in the same script or imported elsewhere):
if __name__ == '__main__':
    # Import the function from the previous script
    def calculate_features(df):
        """Engineers all necessary features from the raw OHLCV data."""
        df = df.copy()
        
        # 1. Relative Strength Index (RSI) - Fixed to handle division by zero
        delta = df['spy_close'].diff()  # Use spy_close instead of 'Close'
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        
        # Handle division by zero and NaN values
        rs = np.where((loss == 0) | (loss.isna()), np.nan, gain / loss)
        df['rsi'] = 100 - (100 / (1 + rs))
        
        # 2. Bollinger Bands
        df['sma_20'] = df['spy_close'].rolling(window=20).mean()
        df['std_20'] = df['spy_close'].rolling(window=20).std()
        df['bollinger_upper'] = df['sma_20'] + (df['std_20'] * 2)
        df['bollinger_lower'] = df['sma_20'] - (df['std_20'] * 2)

        # 3. Rolling Z-Score
        df['z_score_20'] = (df['spy_close'] - df['sma_20']) / df['std_20']

        # 4. Average True Range (ATR) - Fixed column references
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['spy_close'].shift())  # Use spy_close
        low_close = np.abs(df['Low'] - df['spy_close'].shift())    # Use spy_close
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        df['atr'] = tr.rolling(window=14).mean()
        
        return df

    def get_market_data(start_date="2005-01-01"):
        """Downloads SPY and VIX data and engineers features."""
        import yfinance as yf
        import numpy as np
        
        spy_data = yf.download('SPY', start=start_date, auto_adjust=True)
        vix_data = yf.download('^VIX', start=start_date, auto_adjust=True)

        # Reset any potential multi-level indexes and ensure clean column structure
        if isinstance(spy_data.columns, pd.MultiIndex):
            spy_data.columns = spy_data.columns.droplevel(1)  # Remove the second level if it exists
        if isinstance(vix_data.columns, pd.MultiIndex):
            vix_data.columns = vix_data.columns.droplevel(1)  # Remove the second level if it exists
        
        # Select only the columns we need
        spy_data = spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        vix_data = vix_data[['Close']].copy()
        vix_data = vix_data.rename(columns={'Close': 'vix'})
        
        # Align the date indices properly
        common_dates = spy_data.index.intersection(vix_data.index)
        spy_data = spy_data.loc[common_dates]
        vix_data = vix_data.loc[common_dates]
        
        # Combine into a single DataFrame using merge
        df = spy_data.copy()
        df['vix'] = vix_data['vix']
        df.rename(columns={'Close': 'spy_close'}, inplace=True)  # Rename for clarity

        # Calculate features using the function
        df = calculate_features(df)
        
        # Calculate daily returns for the backtest
        df['daily_return'] = df['spy_close'].pct_change()

        return df.dropna()

    market_data = get_market_data()
    # Use default parameters for now
    signals = generate_atr_target_signals(market_data)
    
    # Check how many signals were generated
    print("Signal distribution:")
    print(signals.value_counts())

    # Plot RSI with signals
    import matplotlib.pyplot as plt
    
    market_data['signal'] = signals
    
    plt.figure(figsize=(15, 7))
    plt.plot(market_data.index, market_data['spy_close'], label='SPY Close')
    plt.scatter(market_data[market_data['signal'] == 1].index, market_data[market_data['signal'] == 1]['spy_close'], marker='^', color='g', s=100, label='Buy Signal')
    plt.scatter(market_data[market_data['signal'] == -1].index, market_data[market_data['signal'] == -1]['spy_close'], marker='v', color='r', s=100, label='Short Signal')
    plt.legend()
    plt.title('SPY with Volatility-Normalised RSI Signals')
    plt.show()

# Price Target Exit (long only)

In [ ]:
import pandas as pd
import numpy as np

def generate_sma_exit_signals(df, rsi_entry=30, atr_multiplier=1.5, vix_threshold=40):
    """
    Generates signals with an exit condition based on crossing the 20-day SMA.
    """
    signals = pd.DataFrame(index=df.index)
    signals['signal'] = 0.0
    position = 0  # 0: flat, 1: long, -1: short

    # --- ENTRY CONDITIONS (Modified with Volume) ---
    long_entry = (df['rsi'] < rsi_entry) & \
                 (df['spy_close'] < df['bollinger_lower']) & \
                 (df['Volume'] > df['volume_sma_50'] * 1.5) 
    
    short_entry = (df['rsi'] > (100 - rsi_entry)) & \
                  (df['spy_close'] > df['bollinger_upper']) & \
                  (df['Volume'] > df['volume_sma_50'] * 1.5)

    for i in range(1, len(df)):
        # --- EXIT CONDITIONS ---
        # Exit long if price crosses above the 20-day SMA
        if position == 1 and (df['spy_close'].iloc[i] > df['sma_20'].iloc[i]):
            position = 0
        # Exit short if price crosses below the 20-day SMA
        elif position == -1 and (df['spy_close'].iloc[i] < df['sma_20'].iloc[i]):
            position = 0
        
        # --- ENTRY LOGIC ---
        if position == 0:
            if long_entry.iloc[i]:
                position = 1
            elif short_entry.iloc[i]:
                position = 0
        
        signals['signal'].iloc[i] = position

    # VIX Regime Filter
    signals.loc[df['vix'] > vix_threshold, 'signal'] = 0
    
    return signals['signal']

# Example usage (in the same script or imported elsewhere):
if __name__ == '__main__':
    # Import the function from the previous script
    def calculate_features(df):
        """Engineers all necessary features from the raw OHLCV data."""
        df = df.copy()
        
        # 1. Relative Strength Index (RSI) - Fixed to handle division by zero
        delta = df['spy_close'].diff()  # Use spy_close instead of 'Close'
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        
        # Handle division by zero and NaN values
        rs = np.where((loss == 0) | (loss.isna()), np.nan, gain / loss)
        df['rsi'] = 100 - (100 / (1 + rs))
        
        # 2. Bollinger Bands
        df['sma_20'] = df['spy_close'].rolling(window=20).mean()
        df['std_20'] = df['spy_close'].rolling(window=20).std()
        df['bollinger_upper'] = df['sma_20'] + (df['std_20'] * 2)
        df['bollinger_lower'] = df['sma_20'] - (df['std_20'] * 2)

        # 3. Rolling Z-Score
        df['z_score_20'] = (df['spy_close'] - df['sma_20']) / df['std_20']

        # 4. Average True Range (ATR) - Fixed column references
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['spy_close'].shift())  # Use spy_close
        low_close = np.abs(df['Low'] - df['spy_close'].shift())    # Use spy_close
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        df['atr'] = tr.rolling(window=14).mean()

        # 5. Add Volume Moving Average
        df['volume_sma_50'] = df['Volume'].rolling(window=50).mean()
        
        return df

    def get_market_data(start_date="2005-01-01"):
        """Downloads SPY and VIX data and engineers features."""
        import yfinance as yf
        import numpy as np
        
        spy_data = yf.download('SPY', start=start_date, auto_adjust=True)
        vix_data = yf.download('^VIX', start=start_date, auto_adjust=True)

        # Reset any potential multi-level indexes and ensure clean column structure
        if isinstance(spy_data.columns, pd.MultiIndex):
            spy_data.columns = spy_data.columns.droplevel(1)  # Remove the second level if it exists
        if isinstance(vix_data.columns, pd.MultiIndex):
            vix_data.columns = vix_data.columns.droplevel(1)  # Remove the second level if it exists
        
        # Select only the columns we need
        spy_data = spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        vix_data = vix_data[['Close']].copy()
        vix_data = vix_data.rename(columns={'Close': 'vix'})
        
        # Align the date indices properly
        common_dates = spy_data.index.intersection(vix_data.index)
        spy_data = spy_data.loc[common_dates]
        vix_data = vix_data.loc[common_dates]
        
        # Combine into a single DataFrame using merge
        df = spy_data.copy()
        df['vix'] = vix_data['vix']
        df.rename(columns={'Close': 'spy_close'}, inplace=True)  # Rename for clarity

        # Calculate features using the function
        df = calculate_features(df)
        
        # Calculate daily returns for the backtest
        df['daily_return'] = df['spy_close'].pct_change()

        return df.dropna()

    market_data = get_market_data()
    # Use default parameters for now
    signals = generate_sma_exit_signals(market_data)
    
    # Check how many signals were generated
    print("Signal distribution:")
    print(signals.value_counts())

    # Plot RSI with signals
    import matplotlib.pyplot as plt
    
    market_data['signal'] = signals
    
    plt.figure(figsize=(15, 7))
    plt.plot(market_data.index, market_data['spy_close'], label='SPY Close')
    plt.scatter(market_data[market_data['signal'] == 1].index, market_data[market_data['signal'] == 1]['spy_close'], marker='^', color='g', s=100, label='Buy Signal')
    plt.scatter(market_data[market_data['signal'] == -1].index, market_data[market_data['signal'] == -1]['spy_close'], marker='v', color='r', s=100, label='Short Signal')
    plt.legend()
    plt.title('SPY with Volatility-Normalised RSI Signals')
    plt.show()